In [ ]:
#@title Please upload your site list CSV with no headers (check's Google Cache and site directly)
!pip install requests-html lxml_html_clean  # Install the necessary package
from requests_html import HTMLSession # Import HTMLSession
import re
import pandas as pd

from datetime import date
!rm -rf sample_data
print('Please upload csv of sites you would like to check')
from google.colab import files
uploaded = files.upload()
for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(name=fn, length=len(uploaded[fn])))
reportname =  fn
GTM_Code = "ENTER_GTM_CODE " #@param ["GTM-NV92H48"] {allow-input: true}
google_cache = 'http://webcache.googleusercontent.com/search?q=cache:'


def test_site(url):
  confirmed = False
  correct_url = False
  codes = []
  gtm_code_list = []
  try:
    session = HTMLSession()
    r = session.get(google_cache+url+'&strip=0&vwsrc=1')
    correct_url = True
  except:
    print(f"Invalid URL: {url}")
    return "Invalid URL"
  if correct_url:
    gtm_code_list = list(set(re.findall(r'GTM-[A-Z0-9\-]{6,20}', r.text, re.IGNORECASE)))
    print(f"There were {len(gtm_code_list)} GTM tags found for {url}. GTM code(s): {', '.join([str(x) for x in gtm_code_list]) }")
    if GTM_Code in gtm_code_list:
      print(f"Our GTM code was found! on {url}")
      confirmed = True
      codes = ', '.join(gtm_code_list)
    else:
      print(f"Our GTM code was NOT found on {url}.")
      codes = 'No GTM found'
    return codes

def test_site_no_cache(url):
  confirmed = False
  correct_url = False
  codes = []
  gtm_code_list = []
  try:
    session = HTMLSession()
    r = session.get(url)
    correct_url = True
  except:
    print(f"Invalid URL: {url}")
    return "Invalid URL"
  if correct_url:
    gtm_code_list = list(set(re.findall(r'GTM-[A-Z0-9\-]{6,20}', r.text, re.IGNORECASE)))
    print(f"There were {len(gtm_code_list)} GTM tags found for {url}. GTM code(s): {', '.join([str(x) for x in gtm_code_list]) }")
    if GTM_Code in gtm_code_list:
      print(f"Our GTM code was found! on {url}")
      confirmed = True
      codes = ', '.join(gtm_code_list)
    else:
      print(f"Our GTM code was NOT found on {url}.")
      codes = 'No GTM found'
    return codes

def confirm_gtm(df):
  df['verified'] = df.gtm_codes.str.contains(GTM_Code)
  return df


today = date.today().strftime("%m_%d_%y")

df = pd.read_csv(fn,header=None)
df.columns = ['site']
df['gtm_codes'] = df['site'].apply(lambda x: test_site(x))
df = confirm_gtm(df)

cached_df = df.loc[df.gtm_codes != 'No GTM found'].copy()

print("Performing double check against GTM's not found...")
new_df = df.loc[df.gtm_codes == 'No GTM found'].copy()
new_df = new_df.reset_index(drop=True).drop(['gtm_codes', 'verified'], axis=1)
new_df['gtm_codes'] = new_df['site'].apply(lambda x: test_site_no_cache(x))
new_df = confirm_gtm(new_df)

print("Combining final results into single dataframe...")
final_df = pd.concat([cached_df, new_df]).reset_index(drop=True)

final_df.to_csv(f"{fn.split('.')[0]}_gtm_check{today}.csv")

!zip -r /content/GTMcheck.zip /content/
files.download("/content/GTMcheck.zip")

final_df
### OLD TEST CODE ###
# import requests
# from bs4 import BeautifulSoup

# def gtm_return(code):
#   return re.search("GTM-[A-Z0-9]{6,7}",str(code)).group(0)

# def test_site(url):
#   confirmed = False
#   correct_url = False
#   codes = []
#   try:
#     user_agent = 'Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2228.0 Safari/537.36'
#     headers = {'User-Agent' : user_agent}
#     req = requests.get(url, headers)
#     correct_url = True
#   except:
#     print(f"Invalid URL: {url}")
#     return "Invalid URL"
#   if correct_url:
#     soup = BeautifulSoup(req.content, 'html.parser')
#     GTM = soup.head.findAll(text=re.compile(r'GTM'))
#     gtm_code_list = [gtm_return(GTM) for code in GTM]
#     print(f"There were {len(GTM)} GTM tags found for {url}. GTM code(s): {', '.join([str(x) for x in gtm_code_list]) }")
#     if GTM_Code in gtm_code_list:
#       print(f"Our GTM code was found! on {url}")
#       confirmed = True
#       codes = ', '.join(gtm_code_list)
#     else:
#       print(f"Our GTM code was NOT found on {url}.")
#       codes = 'No GTM found'
#     return codes